In [1]:
# Create concepts using wikipedia (and its knowledge graph) 
# https://en.wikipedia.org/wiki/List_of_sports

# Search of sports: Found a link "List of sports"

In [1]:
import pandas as pd
import pyarrow as pa
from datasets import load_from_disk, Dataset
import os

In [2]:
path_dir = '/work3/s174498/concept_random_dataset/wikipedia_20220301/train'
wikipedia = load_from_disk(path_dir)

titles = wikipedia['title']

# Create datasets for sports concepts

Acrobatic sports: No 'Main articles'. Thus, all sub-articles are concatenated into one Level II concept, while these sub-articles (if longer than 150 sentences) are saved for Level III concepts.

In [32]:
acrobatic_sports = ['Ballet',
'Breakdancing',
'Competitive dancing',
'Cheerleading',
'Dancesport',
'Dragon dance',
'Figure skating',
'Freerunning',
'Gymnastics',
'High kick',
'Parkour',
'Pole sports',
'Stunt',
'Trampolining',
'Winter guard']

In [33]:
list_of_interest = acrobatic_sports
folder_name = 'Acrobatic sports'

count_datasets = 0
concept_list = []
concept_list_saved = []
concatenate_phrases = []

for i in range(len(list_of_interest)): 
    if list_of_interest[i] in titles:
        concept_list.append(list_of_interest[i])
        idx = titles.index(list_of_interest[i])

        # Clean up data (\n references in wiki article)
        text_clean = wikipedia[idx]["text"].replace('\n\n', ' ')
        text_clean2 = text_clean.replace('\n', ' ')
        phrase_to_list = text_clean2.split(". ")

        # Save concepts to "Level II"
        concatenate_phrases = concatenate_phrases + phrase_to_list

        # Save concepts to "Level III"
        if len(phrase_to_list) > 150:
            print(list_of_interest[i])
            concept_list_saved.append(list_of_interest[i])

            df = pd.DataFrame({'id': wikipedia[idx]["id"], 'url': wikipedia[idx]["url"], 
            'title': wikipedia[idx]["title"], 'text': wikipedia[idx]["text"], 'text_list': phrase_to_list})
            data_concept = Dataset(pa.Table.from_pandas(df))

            # save data
            path = os.path.join('/work3/s174498/concept_random_dataset/wikipedia_20220301/sports_concepts/', folder_name)
            data_concept.save_to_disk(path + '/' + list_of_interest[i])

if len(concatenate_phrases) > 150:
    df_levelII = pd.DataFrame({'title': folder_name, 'text_list': concatenate_phrases})
    data_concept_levelII = Dataset(pa.Table.from_pandas(df_levelII))

    # save data
    data_concept_levelII.save_to_disk('/work3/s174498/concept_random_dataset/wikipedia_20220301/sports_concepts/' + folder_name)

Ballet
Breakdancing
Cheerleading
Figure skating
Gymnastics
Parkour


Next, try all conceps having a 'main articles'

In [34]:
list_of_sport = ['Air sports',
'Archery', 'Boardsport']

In [35]:
list_of_interest = list_of_sport

count_datasets = 0
concept_list = []
concept_list_saved = []
concept_list_notsaved = []

for i in range(len(list_of_interest)): 
    if list_of_interest[i] in titles:
        concept_list.append(list_of_interest[i])
        idx = titles.index(list_of_interest[i])

        # Clean up data (\n references in wiki article)
        text_clean = wikipedia[idx]["text"].replace('\n\n', ' ')
        text_clean2 = text_clean.replace('\n', ' ')
        phrase_to_list = text_clean2.split(". ")
        print(list_of_interest[i])
        print(len(phrase_to_list))

        if len(phrase_to_list) > 150:
            count_datasets = count_datasets + 1
            concept_list_saved.append(list_of_interest[i])

            df = pd.DataFrame({'id': wikipedia[idx]["id"], 'url': wikipedia[idx]["url"], 
            'title': wikipedia[idx]["title"], 'text': wikipedia[idx]["text"], 'text_list': phrase_to_list})
            data_concept = Dataset(pa.Table.from_pandas(df))

            # save data
            data_concept.save_to_disk('/work3/s174498/concept_random_dataset/wikipedia_20220301/sports_concepts/' + list_of_interest[i])
        else: 
            concept_list_notsaved.append(list_of_interest[i])
    else: 
        print(list_of_interest[i] + ' does not exist')

Air sports
5
Archery
322
Boardsport
71


In [36]:
concept_list_notsaved

['Air sports', 'Boardsport']

Air sports: Not enought sentences in 'Main articles'. Thus, all sub-articles are concatenated into one Level II concept, while these sub-articles (if longer than 150 sentences) are saved for Level III concepts.

In [43]:
Air_sports = ['Aerobatics',
'Air racing',
'Gliding',
'Hang gliding',
'Human powered aircraft',
'Model aircraft',
'Parachuting',
'Paragliding',
'Paramotoring',
'Ultralight aviation']

In [44]:
list_of_interest = Air_sports
folder_name = 'Air sports'

count_datasets = 0
concept_list = []
concept_list_saved = []
concept_list_notsaved = []
concatenate_phrases = []

for i in range(len(list_of_interest)): 
    if list_of_interest[i] in titles:
        concept_list.append(list_of_interest[i])
        idx = titles.index(list_of_interest[i])

        # Clean up data (\n references in wiki article)
        text_clean = wikipedia[idx]["text"].replace('\n\n', ' ')
        text_clean2 = text_clean.replace('\n', ' ')
        phrase_to_list = text_clean2.split(". ")

        # Save concepts to "Level II"
        concatenate_phrases = concatenate_phrases + phrase_to_list

        # Save concepts to "Level III"
        if len(phrase_to_list) > 150:
            print(list_of_interest[i])
            concept_list_saved.append(list_of_interest[i])

            df = pd.DataFrame({'id': wikipedia[idx]["id"], 'url': wikipedia[idx]["url"], 
            'title': wikipedia[idx]["title"], 'text': wikipedia[idx]["text"], 'text_list': phrase_to_list})
            data_concept = Dataset(pa.Table.from_pandas(df))

            # save data
            path = os.path.join('/work3/s174498/concept_random_dataset/wikipedia_20220301/sports_concepts/', folder_name)
            data_concept.save_to_disk(path + '/' + list_of_interest[i])
        else: 
            concept_list_notsaved.append(list_of_interest[i])
    else: 
        print(list_of_interest[i] + ' does not exist')

if len(concatenate_phrases) > 150:
    df_levelII = pd.DataFrame({'title': folder_name, 'text_list': concatenate_phrases})
    data_concept_levelII = Dataset(pa.Table.from_pandas(df_levelII))

    # save data
    data_concept_levelII.save_to_disk('/work3/s174498/concept_random_dataset/wikipedia_20220301/sports_concepts/' + folder_name)

Gliding
Hang gliding
Human powered aircraft does not exist
Model aircraft
Parachuting
Paragliding
Paramotoring does not exist


In [45]:
concept_list_notsaved

['Aerobatics', 'Air racing', 'Ultralight aviation']

In [46]:
Boardsport = ['Skateboarding',
'Scootering',
'Skysurfing',
'Streetluge',
'Snowboarding',
'Surfing',
'Wakeboarding',
'Paddleboarding']

In [47]:
list_of_interest = Boardsport
folder_name = 'Boardsport'

count_datasets = 0
concept_list = []
concept_list_saved = []
concept_list_notsaved = []
concatenate_phrases = []

for i in range(len(list_of_interest)): 
    if list_of_interest[i] in titles:
        concept_list.append(list_of_interest[i])
        idx = titles.index(list_of_interest[i])

        # Clean up data (\n references in wiki article)
        text_clean = wikipedia[idx]["text"].replace('\n\n', ' ')
        text_clean2 = text_clean.replace('\n', ' ')
        phrase_to_list = text_clean2.split(". ")

        # Save concepts to "Level II"
        concatenate_phrases = concatenate_phrases + phrase_to_list

        # Save concepts to "Level III"
        if len(phrase_to_list) > 150:
            print(list_of_interest[i])
            concept_list_saved.append(list_of_interest[i])

            df = pd.DataFrame({'id': wikipedia[idx]["id"], 'url': wikipedia[idx]["url"], 
            'title': wikipedia[idx]["title"], 'text': wikipedia[idx]["text"], 'text_list': phrase_to_list})
            data_concept = Dataset(pa.Table.from_pandas(df))

            # save data
            path = os.path.join('/work3/s174498/concept_random_dataset/wikipedia_20220301/sports_concepts/', folder_name)
            data_concept.save_to_disk(path + '/' + list_of_interest[i])
        else: 
            concept_list_notsaved.append(list_of_interest[i])
    else: 
        print(list_of_interest[i] + ' does not exist')

if len(concatenate_phrases) > 150:
    df_levelII = pd.DataFrame({'title': folder_name, 'text_list': concatenate_phrases})
    data_concept_levelII = Dataset(pa.Table.from_pandas(df_levelII))

    # save data
    data_concept_levelII.save_to_disk('/work3/s174498/concept_random_dataset/wikipedia_20220301/sports_concepts/' + folder_name)

Skateboarding
Streetluge does not exist
Snowboarding
Surfing


In [48]:
concept_list_notsaved

['Scootering', 'Skysurfing', 'Wakeboarding', 'Paddleboarding']

In [ ]:
list_of_gender = ['Woman', 'Female', 'Intersex', 'Transsexual', 'Male', 'Man']

In [5]:
list_of_gender = [
'Agender',
'Androgyne',
'Androgynous',
'Bakla',
'Bigender',
'Binary',
'Bissu',
'Butch',
'Calabai',
'Calalai',
'Cisgender',
'Demiflux',
'Demigender',
'Dual gender',
'Endosex',
'Female',
'Femme',
'Gender bender',
'Gender diverse',
'Genderfluid',
'Genderflux',
'Genderfuck',
'Genderless',
'Gender nonconforming',
'Genderqueer',
'Gender variant',
'Intergender',
'Intersex',
'Hijra',
'Kathoey',
'Male',
'Man',
'Multigender',
'Muxe',
'Neurogender',
'Neutrois',
'Non-binary',
'Omnigender',
'Pangender',
'Polygender',
'Third gender',
'Trans man',
'Trans woman',
'Transgender',
'Transfeminine',
'Transmasculine',
'Transsexual',
'Travesti',
'Trigender',
'Two spirit',
'Woman',
'X-gender',
'Xenogender']

In [24]:
list_of_interest = list_of_gender

count_datasets = 0
concept_list = []
concept_list_saved = []

for i in range(len(list_of_interest)): 
    if list_of_interest[i] in titles:
        concept_list.append(list_of_interest[i])
        idx = titles.index(list_of_interest[i])

        # Clean up data (\n references in wiki article)
        text_clean = wikipedia[idx]["text"].replace('\n\n', ' ')
        text_clean2 = text_clean.replace('\n', ' ')
        phrase_to_list = text_clean2.split(". ")
        print(list_of_interest[i])
        print(len(phrase_to_list))
        #if len(phrase_to_list) > 150:
        count_datasets = count_datasets + 1
        concept_list_saved.append(list_of_interest[i])

        df = pd.DataFrame({'id': wikipedia[idx]["id"], 'url': wikipedia[idx]["url"], 
        'title': wikipedia[idx]["title"], 'text': wikipedia[idx]["text"], 'text_list': phrase_to_list})
        data_concept = Dataset(pa.Table.from_pandas(df))

        # save data
        data_concept.save_to_disk('/work3/s174498/concept_random_dataset/wikipedia_20220301/gender_concepts/' + list_of_interest[i])

Woman
225
Female
69
Man
57
Male
60
Intersex
237
Transsexual
220


### Concatenate 'Woman' and 'female'

In [4]:
idx_woman = titles.index('Woman')
idx_female = titles.index('Female')
        
# Clean up data (\n references in wiki article)
text_clean = wikipedia[idx_woman]["text"].replace('\n\n', ' ')
text_clean2 = text_clean.replace('\n', ' ')
w_phrase_to_list = text_clean2.split(". ")

text_clean = wikipedia[idx_female]["text"].replace('\n\n', ' ')
text_clean2 = text_clean.replace('\n', ' ')
f_phrase_to_list = text_clean2.split(". ")

concatenate_phrases = w_phrase_to_list + f_phrase_to_list

df = pd.DataFrame({'title': "woman/female", 'text_list': concatenate_phrases})
data_concept = Dataset(pa.Table.from_pandas(df))

# save data
data_concept.save_to_disk('/work3/s174498/concept_random_dataset/wikipedia_20220301/gender_concepts/' + 'woman_female')


In [66]:
df

,title,text_list
0,woman/female,A woman is an adult female human
1,woman/female,"Prior to adulthood, a female human is referred..."
2,woman/female,The plural women is sometimes used in certain ...
3,woman/female,"Typically, women have two X chromosomes and ar..."
4,woman/female,Female anatomy is distinguished from male anat...
...,...,...
289,woman/female,unrelated genes that are affected by environme...
290,woman/female,This hypothesis is unlikely to apply to a sign...
291,woman/female,Symbol The symbol ♀ (Unicode: U+2640 Alt codes...
292,woman/female,Joseph Justus Scaliger once speculated that th...


### Concatenate 'Man' and 'Male'

In [5]:
idx_man = titles.index('Man')
idx_male = titles.index('Male')
        
# Clean up data (\n references in wiki article)
text_clean = wikipedia[idx_man]["text"].replace('\n\n', ' ')
text_clean2 = text_clean.replace('\n', ' ')
man_phrase_to_list = text_clean2.split(". ")

text_clean = wikipedia[idx_male]["text"].replace('\n\n', ' ')
text_clean2 = text_clean.replace('\n', ' ')
male_phrase_to_list = text_clean2.split(". ")

concatenate_phrases = man_phrase_to_list + male_phrase_to_list

df = pd.DataFrame({'title': "man/male", 'text_list': concatenate_phrases})
data_concept = Dataset(pa.Table.from_pandas(df))

# save data
data_concept.save_to_disk('/work3/s174498/concept_random_dataset/wikipedia_20220301/gender_concepts/' + 'man_male')


In [68]:
df

,title,text_list
0,man/male,A man is an adult human male
1,man/male,"Prior to adulthood, a male human is referred t..."
2,man/male,"Like most other male mammals, a man's genome u..."
3,man/male,Sex differentiation of the male fetus is gover...
4,man/male,"During puberty, hormones which stimulate andro..."
...,...,...
112,man/male,"In seed plants, which exhibit alternation of g..."
113,man/male,"In mammals, including humans, males are typica..."
114,man/male,"In humans, males have more body hair and muscl..."
115,man/male,"In birds, the male often exhibits a colorful p..."
